In [1]:
import os
import numpy as np
import json
import csv

In [2]:
#NOTE: This notebook is very similar to generate_demonstrator_proficiency_splits.ipynb in data_processing/ego_exo. Remove this script before releasing codebase.
#NOTE: First, compare against that notebook and make any necessary changes to that notebook.

In [3]:
ego_exo_data_dir = "/vision/asomaya1/ego_exo/data" #replace with directory to data

train_file = "demonstrator_arxiv24_train.json"
val_file = "demonstrator_arxiv24_val.json"
test_file = "demonstrator_arxiv24_test.json"
is_v2 = True if "24" in train_file else False
v2_downsampled_rel_path = "downscaled/448"

train_path = os.path.join(ego_exo_data_dir, train_file)
val_path = os.path.join(ego_exo_data_dir, val_file)
test_path = os.path.join(ego_exo_data_dir, test_file)

In [4]:
with open(train_path, "rb") as f:
    train_data = json.load(f)

with open(val_path, "rb") as f:
    val_data = json.load(f)

with open(test_path, "rb") as f:
    test_data = json.load(f)

In [5]:
#create demonstrator proficiency annotation encoding:
train_labels = set([elem['proficiency_score'] for elem in train_data])
val_labels = set([elem['proficiency_score'] for elem in val_data])
test_labels = set([elem['proficiency_score'] for elem in test_data])
demonstrator_proficiency_labels = train_labels.union(val_labels).union(test_labels)
label_map = {'Novice': 0, 'Early Expert': 3, 'Intermediate Expert': 2, 'Late Expert': 1}

In [6]:
#ego + exo view names
camera_views = ['ego', 'exo1', 'exo2', 'exo3', 'exo4']

resolution = "takes_448pFull"
if is_v2:
    resolution += "_v2"

In [7]:
#create train split
train_data_paths = {}
all_exo_train_data_paths = []
for view in camera_views:
    train_data_paths[view] = []
for elem in train_data:
    label = label_map[elem['proficiency_score']]
    video_paths = elem['video_paths']
    for view in camera_views:
        path = video_paths[view].replace("takes", resolution)
        if is_v2: #fixes formatting issue in arxiv24 files
            path = path.split("/", maxsplit=1)[1]
            parts = path.split("/")
            directory_to_find = "frame_aligned_videos"
            index = parts.index(directory_to_find) + 1
            parts.insert(index, v2_downsampled_rel_path)
            path = "/".join(parts)
        train_data_paths[view].append((path, label))
        if view != 'ego':
            all_exo_train_data_paths.append((path, label))

In [8]:
#create val split
val_data_paths = {}
all_exo_val_data_paths = []
for view in camera_views:
    val_data_paths[view] = []
for elem in val_data:
    label = label_map[elem['proficiency_score']]
    video_paths = elem['video_paths']
    for view in camera_views:
        path = video_paths[view].replace("takes", resolution)
        if is_v2: #fixes formatting issue in arxiv24 files
            path = path.split("/", maxsplit=1)[1]
            parts = path.split("/")
            directory_to_find = "frame_aligned_videos"
            index = parts.index(directory_to_find) + 1
            parts.insert(index, v2_downsampled_rel_path)
            path = "/".join(parts)
        val_data_paths[view].append((path, label))
        if view != 'ego':
            all_exo_val_data_paths.append((path, label))

In [9]:
#create test split
test_data_paths = {}
all_exo_test_data_paths = []
for view in camera_views:
    test_data_paths[view] = []
for elem in test_data:
    label = label_map[elem['proficiency_score']]
    video_paths = elem['video_paths']
    for view in camera_views:
        path = video_paths[view].replace("takes", resolution)
        if is_v2: #fixes formatting issue in arxiv24 files
            path = path.split("/", maxsplit=1)[1]
            parts = path.split("/")
            directory_to_find = "frame_aligned_videos"
            index = parts.index(directory_to_find) + 1
            parts.insert(index, v2_downsampled_rel_path)
            path = "/".join(parts)
        test_data_paths[view].append((path, label))
        if view != 'ego':
            all_exo_test_data_paths.append((path, label))

In [10]:
# Save splits to csv
for view in camera_views:
    view_path = os.path.join(resolution.split("_", maxsplit=1)[-1], view)
    if not os.path.exists(view_path):
        os.makedirs(view_path)

for view in camera_views:
    view_path = os.path.join(resolution.split("_", maxsplit=1)[-1], view)
    with open(os.path.join(view_path, "train.csv"), "w", newline="") as csv_file:
        csv_writer = csv.writer(csv_file)
        for item in train_data_paths[view]:
            csv_writer.writerow([f"{item[0]} {item[1]}"])

for view in camera_views:
    view_path = os.path.join(resolution.split("_", maxsplit=1)[-1], view)
    with open(os.path.join(view_path, "val.csv"), "w", newline="") as csv_file:
        csv_writer = csv.writer(csv_file)
        for item in val_data_paths[view]:
            csv_writer.writerow([f"{item[0]} {item[1]}"])

for view in camera_views:
    view_path = os.path.join(resolution.split("_", maxsplit=1)[-1], view)
    with open(os.path.join(view_path, "test.csv"), "w", newline="") as csv_file:
        csv_writer = csv.writer(csv_file)
        for item in test_data_paths[view]:
            csv_writer.writerow([f"{item[0]} {item[1]}"])

In [11]:
if not os.path.exists(os.path.join(resolution.split("_", maxsplit=1)[-1], "exo_all")):
        os.makedirs(os.path.join(resolution.split("_", maxsplit=1)[-1], "exo_all"))

#save all_exo splits to csv
with open(os.path.join(resolution.split("_", maxsplit=1)[-1], "exo_all", "train.csv"), "w", newline="") as csv_file:
    csv_writer = csv.writer(csv_file)
    for item in all_exo_train_data_paths:
        csv_writer.writerow([f"{item[0]} {item[1]}"])

with open(os.path.join(resolution.split("_", maxsplit=1)[-1], "exo_all", "val.csv"), "w", newline="") as csv_file:
    csv_writer = csv.writer(csv_file)
    for item in all_exo_val_data_paths:
        csv_writer.writerow([f"{item[0]} {item[1]}"])

with open(os.path.join(resolution.split("_", maxsplit=1)[-1], "exo_all", "test.csv"), "w", newline="") as csv_file:
    csv_writer = csv.writer(csv_file)
    for item in all_exo_test_data_paths:
        csv_writer.writerow([f"{item[0]} {item[1]}"])